<a href="https://colab.research.google.com/github/Wayne122/Content_Statistics_Demo/blob/main/Blastoff_Content_Statistics_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prerequisite

In [1]:
%%bash
git clone https://github.com/Wayne122/Content_Statistics_Demo.git
cd Content_Statistics_Demo
pip install -r requirements.txt
python3 -m spacy download en_core_web_lg

  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-cp36-none-any.whl size=829180944 sha256=3b8d91e7219b7342cfe4c2dbb6fd947fc6f145a00a709ec039260010909e5bd2
  Stored in directory: /tmp/pip-ephem-wheel-cache-8x4lf82l/wheels/2a/c1/a6/fc7a877b1efca9bc6a089d6f506f16d3868408f9ff89f8dbfc
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


Cloning into 'Content_Statistics_Demo'...


## Import module

In [4]:
# Restart runtime before running this cell
%cd Content_Statistics_Demo/
from Blastoff_Content_Statistics import BlastoffContentStatistics

/content/Content_Statistics_Demo


/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


##Loading data

In [5]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

downloaded = drive.CreateFile({'id':"1sFMftHLILNfuS0kpTlsbUzWpSNs3WkeD"})
downloaded.GetContentFile('liar_plus_dataset.zip')

In [6]:
from zipfile import ZipFile

In [7]:
with ZipFile('liar_plus_dataset.zip', 'r') as myzip:
    train_data = myzip.open('train2.tsv')
    test_data = myzip.open('test2.tsv')
    valid_data = myzip.open('val2.tsv')

train_p_df = pd.read_csv(train_data, sep='\t', header=None).drop([0], axis=1).dropna(how='all')
test_p_df = pd.read_csv(test_data, sep='\t', header=None).drop([0], axis=1).dropna(how='all')
valid_p_df = pd.read_csv(valid_data, sep='\t', header=None).drop([0], axis=1).dropna(how='all')

## Rename the columns

as the module requires the data to have "Statement" and "Label" to train. If you just want to inference, then "Statement" is enough.

In [8]:
train_p_df.rename(columns={2:'Label', 3:'Statement'}, inplace=True)
test_p_df.rename(columns={2:'Label', 3:'Statement'}, inplace=True)

In [9]:
X_train = train_p_df.drop('Label', axis=1)
y_train = train_p_df['Label']
X_test = test_p_df.drop('Label', axis=1)
y_test = test_p_df['Label']

## Initialize module

In [12]:
bcs = BlastoffContentStatistics()

Loading default pretrained model.


## Extract feature

In [ ]:
bcs.extract(X_train)

Model not found: here.pickle


,CC,CD,DT,EX,FW,IN,JJ,LS,MD,NN,PDT,POS,PRP,RB,RP,SYM,TO,UH,VB,WH,SA_C,SA_P,SA_NU,SA_NG,NER_pers,NER_norp,NER_fac,NER_org,NER_gpe,NER_loc,NER_prod,NER_eve,NER_woa,NER_law,NER_lang,NER_dat,NER_tim,NER_per,NER_mon,NER_quan,NER_ordi,NER_cardi
0,0,0,1,0,0,1,1,0,0,6,0,0,0,0,0,1,0,0,2,0,0.2500,0.192,0.692,0.115,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,0,0,1,0,0,3,1,0,0,9,0,0,1,0,1,4,1,0,5,2,0.3612,0.098,0.902,0.000,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,2,0,0,4,0,0,0,9,0,0,0,0,0,3,1,0,3,0,0.3182,0.206,0.687,0.107,3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,2,0,0,7,0,0,0,0,0,1,1,0,2,0,0.7579,0.394,0.606,0.000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,2,0,0,2,1,0,0,3,0,0,1,0,0,1,0,0,1,0,0.0000,0.000,1.000,0.000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10237,0,0,1,2,0,4,1,0,0,7,0,0,0,0,0,1,0,0,2,0,-0.7506,0.062,0.619,0.319,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
10238,1,0,2,0,0,2,0,0,0,8,0,0,0,1,0,1,0,0,2,0,0.4019,0.172,0.828,0.000,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
10239,0,0,2,0,0,5,2,0,1,11,0,0,0,0,0,3,1,0,5,1,0.5859,0.161,0.839,0.000,0,0,0,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0
10240,1,0,1,0,0,1,0,0,0,5,0,0,0,0,0,1,1,0,2,0,0.0000,0.000,1.000,0.000,0,1,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0


## Train model

In [ ]:
bcs.fit(X_train, y_train, X_test, y_test)

Classifier = Nearest Neighbors, Score (test, accuracy) = 20.68, Training time = 0.56 seconds
Classifier = Linear SVM, Score (test, accuracy) = 25.49, Training time = 58.24 seconds
Classifier = RBF SVM, Score (test, accuracy) = 21.15, Training time = 119.31 seconds
Classifier = Decision Tree, Score (test, accuracy) = 23.76, Training time = 0.03 seconds
Classifier = Random Forest, Score (test, accuracy) = 24.55, Training time = 0.04 seconds
Classifier = Neural Net, Score (test, accuracy) = 24.23, Training time = 14.78 seconds
Classifier = AdaBoost, Score (test, accuracy) = 24.23, Training time = 0.74 seconds
Classifier = Naive Bayes, Score (test, accuracy) = 8.52, Training time = 0.01 seconds
Classifier = QDA, Score (test, accuracy) = 7.58, Training time = 0.08 seconds
--------------------------------------------------------------------------------
Best --> Classifier = Linear SVM, Score (test, accuracy) = 25.49


/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


## Export the trained model

In [ ]:
bcs.export('/content/')

## Inferencing

In [13]:
t = bcs.predict(X_test)

In [14]:
t

,Label
0,false
1,false
2,false
3,false
4,half-true
...,...
1262,false
1263,false
1264,false
1265,false


## Generate possibilities

In [15]:
prob_test = bcs.clf.predict_proba(bcs.extract(X_test))

In [17]:
prob_test

array([[0.08203474, 0.21894051, 0.14794616, 0.18739633, 0.19645298,
        0.16722929],
       [0.08208278, 0.21521544, 0.14954956, 0.1961557 , 0.18587003,
        0.1711265 ],
       [0.08246543, 0.23984141, 0.18759866, 0.20146431, 0.14983595,
        0.13879424],
       ...,
       [0.08235875, 0.19065644, 0.18281095, 0.20882673, 0.18085837,
        0.15448877],
       [0.08235624, 0.20778048, 0.18379886, 0.20417851, 0.16247084,
        0.15941508],
       [0.08173541, 0.1952643 , 0.16854824, 0.20582555, 0.20163806,
        0.14698844]])

In [18]:
bcs.clf.classes_

array([0, 1, 2, 3, 4, 5])